In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Note for KY - Jon and Zhiyong you can kind of get the context here too
Once we get the dataset to be perfectly fine, lets test against different types of k-means clusters 
- Potential suggestion: Explore the dataset to see if there is any form of PCA needed

Find the number of k with the lowest WSS, so that we're able to cluster them accordingly.
After that, we can use the elbow method to find the best k value for the dataset and cluster them accordingly.

After that, we manually analyse the data and from there we are able to then further group the data into two groups.

In [2]:
app = pd.read_csv("application_with_outcome.csv", index_col='Unnamed: 0')

# Check if the data is balanced or not, if it is not we will have to do something to it
app['outcome'].value_counts()

1    245321
0    193236
Name: outcome, dtype: int64

In [3]:
app.head()

,own_car,own_realty,num_child,income,age,years_of_employment,mobile,work_phone,phone,email,...,With parents,Government,Sales,Unemployed,Unknown,has_previous_credit,count_X,count_C,months_late,outcome
0,1,1,0,12.965712,32,12,1,1,0,0,...,0,0,0,0,1,1,1,13,3,1
1,1,1,0,12.965712,32,12,1,1,0,0,...,0,0,0,0,1,1,1,12,3,1
2,1,1,0,11.630717,58,3,1,0,0,0,...,0,0,0,0,1,1,16,7,7,0
3,0,1,0,12.506181,52,8,1,0,1,1,...,0,0,1,0,0,1,3,0,2,0
4,0,1,0,12.506181,52,8,1,0,1,1,...,0,0,1,0,0,1,5,0,0,0


In [4]:
print(app.columns)
X = app.drop(['outcome'], axis=1)
y = app['outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Index(['own_car', 'own_realty', 'num_child', 'income', 'age',
       'years_of_employment', 'mobile', 'work_phone', 'phone', 'email',
       'num_family_numbers', 'is_female', 'is_male', 'Commercial associate',
       'Pensioner', 'State servant', 'Student', 'Working', 'Academic degree',
       'Higher education', 'Incomplete higher', 'Lower secondary',
       'Secondary / secondary special', 'Civil marriage', 'Married',
       'Separated', 'Single / not married', 'Widow', 'Co-op apartment',
       'House / apartment', 'Municipal apartment', 'Office apartment',
       'Rented apartment', 'With parents', 'Government', 'Sales', 'Unemployed',
       'Unknown', 'has_previous_credit', 'count_X', 'count_C', 'months_late',
       'outcome'],
      dtype='object')


In [5]:
# For each model

# We simplify the training of the model part
def result(X_train, X_test, y_train, model):
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    return y_pred

In [6]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter = 2000)
log_reg_result = result(X_train, X_test, y_train, log_reg)
log_reg_result

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([7.95245922e-93, 2.53799397e-08, 1.00000000e+00, ...,
       3.15523167e-03, 1.00000000e+00, 7.85401053e-93])

In [7]:
# Support Vector Machine

from sklearn import svm
svm_model = svm.SVC(cache_size = 20000)
svm_model_result = result(X_train, X_test, y_train, svm_model)
svm_model_result

AttributeError: predict_proba is not available when  probability=False

In [ ]:
# Decision Tree / Random Forest (alternative)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

dtc = DecisionTreeClassifier()
cross_val_score(dtc, X_train, y_train, cv=10) # sus af tho

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.99996743, 1.        , 0.99996743, 1.        ])

In [ ]:
# XG Boost

from xgboost import XGBClassifier
XG_boost = XGBClassifier(n_estimators = 100, learning_rate = 1.0, max_depth = 1, objective="binary:logistic").fit(X_train, y_train)
XG_boost_result = result(X_train, X_test, y_train, XG_boost)
XG_boost_result

array([1.1839567e-06, 1.1839567e-06, 9.9999928e-01, ..., 1.5535777e-06,
       9.9999917e-01, 1.1839567e-06], dtype=float32)

In [18]:
# After we have the result for the few models we're using, we can optimise the hyperparameters to help us find the best model
# For example, we can use GridSearchCV to optimise the hyperparameters for Logistic Regression.
# How do you find the parameters? Documentation

from sklearn.model_selection import GridSearchCV

# THIS IS AN EXAMPLE. ADD HOW YOU DEEM FIT.
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 1000, 2500, 5000]
}

clf = GridSearchCV(log_reg, param_grid=param_grid)

# After that, save this into a model and run result() again
y_pred = result(X_train, X_test, y_train, clf)
y_pred

KeyboardInterrupt: 

After this, we use the ROC AUC score if there is an imbalance in dataset. We try not to use accuracy.

There are also other scores like Precision and Recall, those are usable as well. 

However, let's look at the formulas for precision and recall:
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)

The cost of having a false positive is higher than a false negative as we can deny the loan to a person who is not a defaulter, but we cannot give a loan to a person who is a defaulter.

Therefore we should seek to minimise FP > minimise FN, and therefore the higher the precision, the better.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Insert your model here
model = None

y_pred = model.predict_proba(X_test)[:, 1]

In [ ]:
thresholds = np.linspace(0,1,101)

scores = []

for t in thresholds:
    actual_positive = (y_test == 1)
    actual_negative = (y_test == 0)

    pred_positive = (y_pred >= t)
    pred_negative = (y_pred < t)

    tp = (actual_positive & pred_positive).sum()
    tn = (actual_negative & pred_negative).sum()
    fp = (pred_positive & actual_negative).sum()
    fn = (pred_negative & actual_positive).sum()

    scores.append((t, tp, fp, fn, tn))
    
columns = ['threshold', 'tp', 'fp', 'fn', 'tn']
scores_data = pd.DataFrame(scores, columns = columns)

In [ ]:
scores_data['tpr'] = scores_data.tp / (scores_data.tp + scores_data.fn)
scores_data['fpr'] = scores_data.fp / (scores_data.fp + scores_data.tn)

plt.plot(scores_data.threshold, scores_data.tpr, label = 'tpr')
plt.plot(scores_data.threshold, scores_data.fpr, label = 'fpr')
plt.legend()

In [ ]:
# Make conclusions on the graph pls 

scores_data['precision'] = scores_data.tp / (scores_data.tp + scores_data.fp)
scores_data['recall'] = scores_data.tp / (scores_data.tp + scores_data.fn)

plt.plot(scores_data.threshold, scores_data.precision, label = 'precision')
plt.plot(scores_data.threshold, scores_data.recall, label = 'recall')
plt.legend()